# Refinitiv Data Platform Library for Python
## Content - Streaming Prices - How to use a real-time data cache

This notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) from the Refinitiv Data Platform. The example shows how to define a StreamingPrices object, which automatically manages a streaming cache available for access at any time. Your application can then reach into this cache and pull out real-time snapshots as Pandas DataFrames by just calling a simple access method.

Using a StreamingPrices object that way prevents your application from sending too many requests to the platform. This is particularly useful if your application needs to retrieve real-time snapshots at regular and short intervals.

#### Learn more

To learn more about the Refinitiv Data Platform Data Libraries just connect to the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/iam/login) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like [Quick Start guides](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/quick-start), [Tutorials](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/learning), [Documentation](https://developers.refinitiv.com/refinitiv-data-platform/refinitiv-data-platform-libraries/docs) and much more.  

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [Refinitiv Data Platform Q&A Forum](https://community.developers.thomsonreuters.com/spaces/231/index.html). The Refinitiv Developer Community will be very pleased to help you. 

## Import the library and load credentials

Credentials used by this notebook are stored in the ./credentials.ipynb. Please edit ./credentials.ipynb to set your credentials and run the next cell to continue with this scripts

In [1]:
import refinitiv.dataplatform as rdp

%run ./credentials.ipynb

## Open the session of your choice

Depending on the access point your application uses to connect to the Refinitiv Data Platform, it needs to call one of the following functions to create and open a session to the platform. This session will become the default session used by all other function calls.

#### Either

Create and open a Desktop session to connect to the Refinitiv Data Platform pvia Eikon 4 or the Refinitiv Workspace.

In [2]:
rdp.open_desktop_session(APP_KEY)

#### or

Create and open a Platform session to connect directly to the Refinitiv Data Platform. 

In [12]:
rdp.open_platform_session(
    APP_KEY, 
    rdp.GrantPassword(
        username = RDP_LOGIN, 
        password = RDP_PASSWORD
    )
)

#### or

Create and open a Deployed Platform session to connect directly to a deployed enterprise platform (a.k.a.TREP). 

In [22]:
rdp.open_deployed_platform_session(
    APP_KEY,
    DEPLOYED_PLATFORM_HOST,
    DEPLOYED_PLATFORM_USER_NAME
)

## Create and open a StreamingPrices object

The StreamingPrices object is created for a list of instruments and fields. The fields parameter is optionnal. If you omit it, StreamingPrices will retrieve all fields available for the requested instruments

In [23]:
streaming_prices = rdp.StreamingPrices(
    universe = ['GBP=', 'JPY=', 'CAD='], 
    fields   = ['BID', 'ASK', 'OPEN_PRC', 'HST_CLOSE']
)

The open method tells the StreamingPrices object to subscribe to the streams of the requested instruments.

In [24]:
streaming_prices.open()

<StreamState.Open: 3>

As soon as the open method returns, the StreamingPrices object is ready to be used. Its internal cache is constantly kept updated with the latest streaming information received from Eikon / Refinitiv Workspace. All this happens behind the scene, waiting for your application to pull out data from the cache.  

## Use get_snapshot to pull out data as a DataFrame
Once the StreamingPrices object is opened, you can use the get_snapshot method to pull out data from its internal cache. get_snapshot returns a Pandas DataFrame that uses the same format as the eikon.get_data() function.

get_snapshot can be called any number of times. As these calls return the latest values received from Eikon, several calls to get_snapshot on the same StreamingPrices object may return DataFrames containing different values. Values of the returned DataFrames do not change in real-time, get_snapshot must be called every time your application needs fresh real-time values. 

In [25]:
df = streaming_prices.get_snapshot()
display(df)

,Instrument,OPEN_PRC,HST_CLOSE,BID,ASK
0,GBP=,1.3077,1.3076,1.3045,1.3049
1,JPY=,109.5400,109.5300,109.4000,109.4300
2,CAD=,1.3110,1.3113,1.3119,1.3120


## Get a snapshot for a subset of instruments and fields

In [26]:
df = streaming_prices.get_snapshot(
    universe = ['JPY=', 'CAD='], 
    fields   = ['BID', 'ASK']
)
display(df)

,Instrument,BID,ASK
0,JPY=,109.4000,109.430
1,CAD=,1.3119,1.312


## Other options to get values from the streaming cache

#### Direct access to real-time fields

In [27]:
print('CAD/BID:', streaming_prices['CAD=']['BID'])
print('JPY/BID:', streaming_prices['JPY=']['BID'])

CAD/BID: 1.3115
JPY/BID: 109.4


#### Direct acces to a Streaming Price instrument

In [28]:
cad = streaming_prices['CAD=']
print(cad.status)
print(cad['BID'])

{'status': <StreamState.Open: 3>, 'code': 'Open', 'message': 'All is well'}
1.3114


#### Iteration on fields

In [29]:
print('CAD=')
for field_name, field_value in streaming_prices['CAD=']:
    print('\t' + field_name + ': ', field_value)
    
print('JPY=')
for field_name, field_value in streaming_prices['JPY=']:
    print('\t' + field_name + ': ', field_value)

CAD=
	OPEN_PRC:  1.311
	HST_CLOSE:  1.3113
	BID:  1.3118
	ASK:  1.3122
JPY=
	OPEN_PRC:  109.54
	HST_CLOSE:  109.53
	BID:  109.42
	ASK:  109.43


#### Iteration on StreamingPrice instruments and fields

In [30]:
for streaming_price in streaming_prices:
    print(streaming_price.name)
    for field_name, field_value in streaming_price:
        print('\t' + field_name + ': ', field_value)

GBP=
	OPEN_PRC:  1.3077
	HST_CLOSE:  1.3076
	BID:  1.3045
	ASK:  1.3049
JPY=
	OPEN_PRC:  109.54
	HST_CLOSE:  109.53
	BID:  109.42
	ASK:  109.43
CAD=
	OPEN_PRC:  1.311
	HST_CLOSE:  1.3113
	BID:  1.3118
	ASK:  1.3122


### Close StreamingPrices

In [31]:
streaming_prices.close()

<StreamState.Closed: 1>

Once closed is called the StreamingPrices object stops updating its internal cache. The get_snapshot function can still be called but after the close it always return the same values. 